In [1]:
import time
from design import *
import importlib
import shutil
from utils import *
from openai import OpenAI
from prompts import *
import json
import numpy as np
from gymnasium.envs.robodesign.GPTHopper import GPTHopperEnv
import os

In [ ]:
import prompts
class DGA:
    def __init__(self):
        self.client = OpenAI(api_key=api_key)
        # self.model = "gpt-3.5-turbo"
        self.model = "gpt-4-turbo"

    def extract_code(self, text):
        match = re.search(r'```python\n(.*?)\n```', text, re.DOTALL)
        return match.group(1).strip() if match else None

    def indent_code(self, code):
        return "\n".join(line if line.strip() else line for line in code.split("\n"))

    def generate_rewardfunc(self, rewardfunc_nums, folder_name):

        messages = [
            {"role": "system", "content": "You are a reinforcement learning reward function designer"},
            {"role": "user", "content": rewardfunc_prompts + zeroshot_rewardfunc_format}
        ]

        responses = self.client.chat.completions.create(
            model=self.model, messages=messages, n=rewardfunc_nums
        )
        files = []
        for i, choice in enumerate(responses.choices):
            reward_code = self.extract_code(choice.message.content)
            if reward_code:
                full_code = self.indent_code(reward_code) + "\n"
                file_name =  f"GPTHopper_{i}.py"
                file_path = os.path.join(folder_name, "env", file_name)
                with open(file_path, "w") as fp:
                    fp.write(full_code)

                with open(file_path, "w") as fp:
                    fp.write(full_code)
                files.append(file_path)
                print(f"Saved: {file_path}")
        return files
    
    def generate_rewardfunc_div(self, rewardfunc_nums, folder_name):

        messages = [
            {"role": "system", "content": "You are a reinforcement learning reward function designer"},
            {"role": "user", "content": rewardfunc_prompts + zeroshot_rewardfunc_format}
        ]

        # 生成初始 Reward Function
        response = self.client.chat.completions.create(
            model=self.model, messages=messages, n=1, timeout=10
        )

        rewardfunc_files = []

        initial_code = self.extract_code(response.choices[0].message.content)
        if initial_code:
            reward_code = "import numpy as np\n" + self.indent_code(initial_code) + "\n"

            file_path = os.path.join(folder_name, "env", "GPTrewardfunc_0.py")
            with open(file_path, "w") as fp:
                fp.write(reward_code)
            rewardfunc_files.append(file_path)
            print(f"initial Saved: {file_path}")
        messages.append({"role": "assistant", "content": initial_code})

        # 生成不同的多样化 Reward Functions
        for i in range(1, rewardfunc_nums):
            diverse_messages = messages + [
                {"role": "user", "content": rewardfunc_div_prompts + zeroshot_rewardfunc_format}
            ]
            # print(diverse_messages)
            response = self.client.chat.completions.create(
                model=self.model, messages=diverse_messages, n=1
            )
            diverse_code = self.extract_code(response.choices[0].message.content)
            messages.append({"role": "assistant", "content": diverse_code})

            if diverse_code:
                reward_code =  "import numpy as np\n" + self.indent_code(diverse_code) + "\n"
                file_path = os.path.join(folder_name, "env", f"GPTrewardfunc_{i}.py")
                with open(file_path, "w") as fp:
                    fp.write(reward_code)
                rewardfunc_files.append(file_path)
                print(f"Saved: {file_path}")

        return rewardfunc_files


    def generate_morphology(self, morphology_nums, folder_name):
        messages = [
            {"role": "system", "content": "You are a helpful mujoco robot designer"},
            {"role": "user", "content": morphology_prompts + morphology_format}
        ]
        
        responses = self.client.chat.completions.create(
            model=self.model,
            messages=messages,
            response_format={'type': 'json_object'},
            n=morphology_nums                                                                                                                                                                                                                                                                                   
        )

        # 解析所有 response 里的参数
        for i, choice in enumerate(responses.choices):
            print(f"Response {i}:")
            print(json.dumps(choice.message.content, indent=4))

        parameter_list = [json.loads(choice.message.content).get('parameters', []) for choice in responses.choices]
        material_list = [compute_hopper_volume(parameter) for parameter in parameter_list]

        xml_files = []
        for i, parameter in enumerate(parameter_list):
            if not isinstance(parameter, list):
                print(f"Skipping invalid parameter {i}: {parameter}")
                continue

            xml_file = hopper_design(parameter)  
            filename = f"GPTHopper_{i}.xml"
            file_path = os.path.join(folder_name, "assets", filename)
            xml_files.append(file_path)
            with open(file_path, "w") as fp:
                fp.write(xml_file)
            print(f"Successfully saved {filename}")
            
        return xml_files, material_list, parameter_list
    
    def generate_morphology_div(self, morphology_nums, folder_name):

        material_list = []
        xml_files = []
        parameter_list = []
        
        # 生成初始 morphology
        messages = [
            {"role": "system", "content": "You are a helpful mujoco robot designer"},
            {"role": "user", "content": morphology_prompts + morphology_format}
        ]
        
        response = self.client.chat.completions.create(
            model=self.model,
            messages=messages,
            response_format={'type': 'json_object'},
            n=1
        )
        

        initial_parameter = json.loads(response.choices[0].message.content)
        parameter_list.append(initial_parameter['parameters'])
        material_list.append(compute_hopper_volume(initial_parameter['parameters']))
        messages.append({"role": "assistant", "content": json.dumps(initial_parameter)})

        logging.info(f"generate initial_parameter{initial_parameter['parameters']}" )

        xml_file = hopper_design(initial_parameter['parameters'])  

        filename = f"GPTHopper_0.xml"
        file_path = os.path.join(folder_name, "assets", filename)
        with open(file_path, "w") as fp:
            fp.write(xml_file)

        xml_files.append(file_path)

        # 生成不同的多样化设计
        for i in range(1, morphology_nums):
            diverse_messages = messages + [
                {"role": "user", "content": morphology_div_prompts + morphology_format}
            ]
            
            response = self.client.chat.completions.create(
                model=self.model,
                messages=diverse_messages,
                response_format={'type': 'json_object'},
                n=1
            )

            diverse_parameter = json.loads(response.choices[0].message.content)
            material_list.append(compute_hopper_volume(diverse_parameter['parameters'])) 
            parameter_list.append(diverse_parameter['parameters'])
            messages.append({"role": "assistant", "content": json.dumps(diverse_parameter)})
            logging.info(f"generate diverse_parameter{ diverse_parameter['parameters']}")
            xml_file = hopper_design(diverse_parameter['parameters'])  
            filename = f"GPTHopper_{i}.xml"
            file_path = os.path.join(folder_name, "assets", filename)
            with open(file_path, "w") as fp:
                fp.write(xml_file)
            xml_files.append(file_path)

        return xml_files, material_list, parameter_list


    def improve_rewardfunc(self, best_rewardfunc, rewardfunc_list, fitness_list, folder_name, step, rewardfunc_index, morphology_index):
        reward_improve_prompts = prompts.reward_improve_prompts

        for rewardfunc_file, fitness in zip(rewardfunc_list, fitness_list):
            with open(rewardfunc_file, "r") as fp:
                reward_content = fp.read()
            reward_improve_prompts += f"\nreward function:\n{reward_content}\nfitness: {fitness}\n"

        with open(best_rewardfunc, "r") as fp:
            best_reward_content = fp.read()
        reward_improve_prompts += f"\nbest reward function:\n{best_reward_content}\nbest fitness: {max(fitness_list)}\n"

        messages = [
            {"role": "system", "content": "You are a reinforcement learning reward function designer"},
            {"role": "user", "content":reward_improve_prompts+ zeroshot_rewardfunc_format}
        ]
        print(messages)
        response = self.client.chat.completions.create(
            model=self.model, messages=messages
        )

        print(response)
        reward_code = self.extract_code(response.choices[0].message.content)

        if reward_code:
            full_code = "import numpy as np \n" + self.indent_code(reward_code) + "\n"
            file_name =  f"GPTrewardfunc_refine_{step}_{rewardfunc_index}_{morphology_index}.py"
            file_path = os.path.join(folder_name, "env", file_name)
            with open(file_path, "w") as fp:
                fp.write(full_code)

        return file_path

    def improve_morphology(self, best_parameter, parameter_list, fitness_list, folder_name, rewardfunc_index, morphology_index, iteration):
        morphology_improve_prompts = prompts.morphology_improve_prompts
        for parameter_content, fitness in zip(parameter_list, fitness_list):
            morphology_improve_prompts = morphology_improve_prompts + f"parameter:{parameter_content} \n" + f"fintess:{fitness}"
        morphology_improve_prompts = morphology_improve_prompts + f"best parameter:{best_parameter} \n" + f"best fintess:{max(fitness_list)}"  

        messages = [
            {"role": "system", "content": "You are a helpful mujoco robot designer"},
            {"role": "user", "content": morphology_improve_prompts + morphology_format}
        ]
        
        responses = self.client.chat.completions.create(
            model=self.model,
            messages=messages,
            response_format={'type': 'json_object'},
        )
        # print(responses)
        parameter = json.loads(responses.choices[0].message.content).get('parameters', []) 
        print(parameter)
        xml_file = hopper_design(parameter)  
        filename = f"GPTHopper_refine_{rewardfunc_index}_{morphology_index}_{iteration}.xml"
        file_path = os.path.join(folder_name, "assets", filename)

        with open(file_path, "w") as fp:
            fp.write(xml_file)

        print(f"Successfully saved {filename}")
        return file_path, parameter


# Configuration

In [3]:
folder_name = "results/Div_m25_r5"
log_file = os.path.join(folder_name, "new_parameters.log")
logging.basicConfig(filename=log_file, level=logging.INFO, format="%(asctime)s - %(message)s")

# folder_name = setup_logging(div_flag=True)

best_fitness = float('-inf')  
best_morphology = None  
best_rewardfunc = None  
best_reward = None
best_material = None
best_efficiency = None

morphology_nums = 26
rewardfunc_nums = 6

fitness_matrix = np.array([[None for _ in range(morphology_nums)] for _ in range(rewardfunc_nums)])
efficiency_matrix = np.array([[None for _ in range(morphology_nums)] for _ in range(rewardfunc_nums)])
fitness_list = []
designer = DGA()


# print configuration info

In [4]:
logging.info(f"start!")

In [5]:
designer = DGA()
morphology_list, material_list, parameter_list = designer.generate_morphology_div(morphology_nums, folder_name)

In [8]:
designer = DGA()
rewardfunc_list = designer.generate_rewardfunc_div(rewardfunc_nums, folder_name)

initial Saved: results/Div_m50_r10\env\GPTrewardfunc_0.py
Saved: results/Div_m50_r10\env\GPTrewardfunc_1.py
Saved: results/Div_m50_r10\env\GPTrewardfunc_2.py
Saved: results/Div_m50_r10\env\GPTrewardfunc_3.py
Saved: results/Div_m50_r10\env\GPTrewardfunc_4.py
Saved: results/Div_m50_r10\env\GPTrewardfunc_5.py
Saved: results/Div_m50_r10\env\GPTrewardfunc_6.py
Saved: results/Div_m50_r10\env\GPTrewardfunc_7.py
Saved: results/Div_m50_r10\env\GPTrewardfunc_8.py
Saved: results/Div_m50_r10\env\GPTrewardfunc_9.py
Saved: results/Div_m50_r10\env\GPTrewardfunc_10.py


In [7]:
parameter_list = [[1.2, 0.9, 0.6, 0.3, 0.3, -0.3, 0.05, 0.04, 0.05, 0.05],
 [1.5, 1.2, 0.8, 0.5, 0.5, -0.5, 0.03, 0.03, 0.03, 0.04],
 [1.0, 0.7, 0.5, 0.2, 0.2, -0.2, 0.04, 0.05, 0.06, 0.07],
 [1.0, 0.6, 0.4, 0.1, 0.1, -0.1, 0.05, 0.04, 0.03, 0.02],
 [1.3, 0.9, 0.4, 0.1, 0.1, -0.1, 0.02, 0.02, 0.04, 0.06],
 [1.8, 1.3, 0.7, 0.2, 0.2, -0.2, 0.01, 0.03, 0.05, 0.06],
 [1.0, 0.8, 0.5, 0.2, 0.18, -0.2, 0.03, 0.02, 0.02, 0.03],
 [1.0, 0.9, 0.6, 0.2, 0.2, -0.2, 0.02, 0.04, 0.06, 0.08],
 [1.5, 1.2, 0.8, 0.4, 0.35, -0.35, 0.02, 0.02, 0.01, 0.05],
 [1.2, 0.8, 0.5, 0.3, 0.25, -0.25, 0.03, 0.05, 0.06, 0.02],
 [1.6, 1.1, 0.7, 0.4, 0.35, -0.35, 0.03, 0.03, 0.03, 0.04],
 [1.4, 0.9, 0.6, 0.2, 0.2, -0.2, 0.01, 0.01, 0.02, 0.04],
 [1.8, 1.1, 0.9, 0.6, 0.55, -0.6, 0.015, 0.015, 0.015, 0.025],
 [1.7, 1.2, 0.8, 0.3, 0.25, -0.25, 0.01, 0.03, 0.04, 0.02],
 [1.0, 0.6, 0.3, 0.1, 0.08, -0.08, 0.015, 0.025, 0.035, 0.045],
 [1.2, 0.5, 0.2, 0.1, 0.12, -0.12, 0.02, 0.015, 0.02, 0.05],
 [1.0, 0.8, 0.2, 0.05, 0.05, -0.05, 0.01, 0.02, 0.03, 0.06],
 [1.8, 0.6, 0.3, 0.05, 0.05, -0.05, 0.025, 0.035, 0.045, 0.02],
 [0.9, 0.4, 0.15, 0.05, 0.05, -0.05, 0.02, 0.03, 0.04, 0.01],
 [1.5, 0.8, 0.5, 0.2, 0.2, -0.2, 0.01, 0.01, 0.01, 0.04],
 [1.4, 0.6, 0.3, 0.1, 0.12, -0.12, 0.015, 0.04, 0.06, 0.02],
 [2.0, 1.0, 0.6, 0.1, 0.15, -0.15, 0.01, 0.02, 0.03, 0.05],
 [0.8, 0.5, 0.4, 0.2, 0.25, -0.25, 0.01, 0.015, 0.02, 0.03],
 [1.1, 0.8, 0.6, 0.2, 0.19, -0.19, 0.015, 0.03, 0.05, 0.02],
 [1.2, 0.4, 0.3, 0.15, 0.15, -0.15, 0.02, 0.04, 0.05, 0.01],
 [1.5, 1.2, 0.9, 0.5, 0.5, -0.5, 0.015, 0.02, 0.025, 0.03],
 [1.0, 0.8, 0.5, 0.1, 0.1, -0.1, 0.02, 0.015, 0.01, 0.06],
 [1.5, 0.7, 0.4, 0.1, 0.1, -0.1, 0.015, 0.025, 0.03, 0.05],
 [1.5, 1.0, 0.4, 0.15, 0.2, -0.2, 0.015, 0.01, 0.035, 0.05],
 [1.5, 1.0, 0.6, 0.3, 0.3, -0.3, 0.015, 0.02, 0.025, 0.03],
 [1.6, 1.2, 0.7, 0.2, 0.25, -0.25, 0.01, 0.04, 0.06, 0.02],
 [1.4, 0.6, 0.4, 0.2, 0.2, -0.35, 0.012, 0.015, 0.018, 0.06],
 [1.0, 0.6, 0.3, 0.05, 0.06, -0.06, 0.01, 0.02, 0.03, 0.05],
 [1.8, 1.0, 0.8, 0.15, 0.15, -0.15, 0.02, 0.02, 0.05, 0.07],
 [1.0, 0.6, 0.25, 0.05, 0.1, -0.1, 0.01, 0.01, 0.04, 0.05],
 [1.0, 0.3, 0.15, 0.05, 0.05, -0.05, 0.015, 0.025, 0.05, 0.1],
 [1.8, 1.2, 0.8, 0.2, 0.15, -0.15, 0.015, 0.04, 0.06, 0.08],
 [1.5, 0.7, 0.3, 0.05, 0.05, -0.05, 0.015, 0.03, 0.07, 0.1],
 [1.8, 1.4, 0.9, 0.3, 0.3, -0.3, 0.012, 0.013, 0.018, 0.065],
 [1.5, 1.0, 0.6, 0.05, 0.05, -0.05, 0.01, 0.02, 0.03, 0.06],
 [1.8, 0.5, 0.2, 0.05, 0.05, -0.05, 0.01, 0.05, 0.1, 0.2],
 [1.2, 0.9, 0.5, 0.1, 0.07, -0.12, 0.02, 0.01, 0.02, 0.08],
 [1.5, 1.0, 0.6, 0.3, 0.35, -0.35, 0.014, 0.018, 0.1, 0.03],
 [1.0, 0.8, 0.5, 0.1, 0.1, -0.1, 0.05, 0.02, 0.01, 0.1],
 [1.2, 0.6, 0.4, 0.15, 0.15, -0.15, 0.015, 0.02, 0.03, 0.04],
 [1.8, 1.4, 1.0, 0.5, 0.6, -0.7, 0.015, 0.02, 0.025, 0.15],
 [1.5, 0.5, 0.2, 0.1, 0.1, -0.15, 0.012, 0.02, 0.04, 0.08],
 [1.4, 0.85, 0.4, 0.05, 0.05, -0.05, 0.015, 0.01, 0.02, 0.03],
 [1.6, 0.5, 0.25, 0.05, 0.05, -0.05, 0.008, 0.015, 0.03, 0.1],
 [1.5, 1.2, 0.6, 0.1, 0.1, -0.3, 0.01, 0.03, 0.05, 0.07],
[1.45, 1.06, 0.6, 0.1, -0.13, 0.26, 0.05, 0.05, 0.04, 0.06]]

morphology_list = [f'results/Div_m25_r5/assets/GPTHopper_{i}.xml' for i in range(0,26) ]
rewardfunc_list = [f'results/Div_m25_r5/env/GPTrewardfunc_{i}.py' for i in range(0,6)]

material_list = [compute_hopper_volume(parameter) for parameter in parameter_list]

# enter coarse optimization stage

In [7]:
logging.info(f'folder_name:{folder_name}')
logging.info(f'morphology_nums:{morphology_nums}')
logging.info(f'rewardfunc_nums:{rewardfunc_nums}')
logging.info(f'parameter_list:{parameter_list}')
logging.info(f'morphology_list:{morphology_list}')
logging.info(f'material_list:{material_list}')
logging.info(f'_________________________________enter coarse optimization stage_________________________________')

In [41]:
for i, rewardfunc in enumerate(rewardfunc_list):
    for j, morphology in enumerate(morphology_list):
        if i not in [10]:
            continue
#         6
        if j not in [50]:
            continue
            
        print(i, rewardfunc)
        print(j, morphology)
        shutil.copy(morphology, "GPTHopper.xml")
        shutil.copy(rewardfunc, "GPTrewardfunc.py")         

        import GPTrewardfunc
        importlib.reload(GPTrewardfunc)  # 重新加载模块
        from GPTrewardfunc import _get_rew
        GPTHopperEnv._get_rew = _get_rew

        # model_path = Train(j,  i, folder_name, total_timesteps=5e5)
        model_path = f"results/Div_m25_r5/coarse/SAC_morphology{j}_rewardfunc{i}_500000.0steps"
        fitness, reward = Eva(model_path)
        material = material_list[j]
        efficiency = fitness/material
        fitness_matrix[i][j] = fitness
        efficiency_matrix[i][j] = efficiency
        
        logging.info("___________________finish coarse optimization_____________________")
        logging.info(f"morphology: {j}, rewardfunc: {i}, material cost: {material} reward: {reward} fitness: {fitness} efficiency: {efficiency}")

        if fitness > best_fitness:
            best_fitness = fitness
            best_morphology = morphology
            best_efficiency = efficiency
            best_rewardfunc = rewardfunc
            best_material = material

10 results/Div_m25_r5/env/GPTrewardfunc_10.py
50 results/Div_m25_r5/assets/GPTHopper_50.xml
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [44]:
efficiency_matrix

array([[None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, 416.00392985310305],
       [None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, 380.17372338336355],
       [None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, 

In [53]:
efficiency_matrix = np.array([[103.21962907117079, 233.56683258847954, 153.97370963015234,
        64.46899994148842, 95.30270715048513, 464.9495760158912,
        5.8866729165511495, 54.16350042537758, 387.5108123794798,
        541.5821962235061, 381.47576916898817, 506.2494067976934,
        351.8947612392126, 755.0536296756648, 490.0052452905147,
        281.2693046575035, 434.44909766727716, 258.93303032707297,
        1594.1496027159913, -48.49147754077618, 513.7156637742906,
        580.399726962935, 282.2402973517378, 3.4135029728225126,
        176.1268148807481],
       [182.7245612197737, 219.6083206292216, 75.2641403292249,
        5.385452461143399, 40.3044785159696, 567.0832799997378,
        1269.6249291661043, -1.9600116490550776, 121.44570746607138,
        780.3050142395712, 330.5631607719396, 288.97271559146947,
        468.9885450915146, 1068.2899662609893, 4.042432386630872,
        314.7953080285454, 119.58990752709401, 243.2625960000366,
        1226.67786710571, 15.03871267870394, 591.6738943873016,
        519.7326324611427, 378.47116809701043, 65.29205638807991,
        205.72804959921424],
       [101.66603472772745, 205.49490339461292, 124.96834028386715,
        399.9205720822782, -58.03690345111858, 357.80289656166923,
        1951.4412234802828, 14.89173422172581, 287.32041781919685,
        369.74567676124815, -5.091345660821362, -49.47618067891138,
        672.6820433619989, 813.8527388443775, 570.455885681394,
        276.2085618477707, 8.800131808654921, 352.35069385334526,
        1162.3153815427256, 591.5133177612977, 64.51706708659094,
        1701.614129629383, 313.35551681845607, -25.979410763984742,
        261.33257961933356],
       [109.95207857109712, 218.47352378199867, 61.37989986715774,
        569.2613709715206, 12.677331644859963, 224.26658069902686,
        -11.37395436154252, 3.1722602441988608, 271.14490526741116,
        598.9978717134203, 17.68395454579057, 492.4054393244021,
        486.1293702617546, 1263.4681637647227, 1497.3660186259829,
        219.36306554752065, 37.59328855899844, 159.09984816725762,
        965.6347490741534, 468.66805808428313, 770.1599459968686,
        522.1544113937665, 337.2843389852064, 23.764182063630514,
        166.07786174798719],
       [105.88197935358606, 215.9686423688591, 19.40079460092955,
        577.7162471797056, 29.57989653386184, 296.8048841883536,
        45.27880522630078, 95.30014860493715, 191.74940297326359,
        540.026098100536, 257.1610882983269, 434.39152267512907,
        346.2299448081738, 788.3289248246854, 229.45096330462232,
        401.50145786102274, 209.88905072592848, 270.78473562860074,
        1427.5921719681094, 514.4583598055287, 866.1259794231802,
        818.8448807620581, 315.1846865645179, 15.642296641033953,
        211.3409659346593]], dtype=object)


efficiency_matrix_select = efficiency_matrix[:5, :25]

# parameter_list[48]
# material_list[48]
# efficiency_matrix_select
mean = np.mean(efficiency_matrix)

std = np.std(efficiency_matrix)

print("平均值：", mean)
print("标准差：", std)


平均值： 375.5161471725431
标准差： 389.74949503144666


In [54]:
fitness_matrix = np.array([[1.3182820167735696, 1.9762921702294036, 2.426997901467287,
        0.3987245659874406, 0.5852316657315135, 5.509693153920175,
        0.015818131281821152, 0.9506252727624258, 2.750514933521477,
        4.306888116094734, 2.868269927638849, 1.5565002563778547,
        1.1310288525040613, 3.6608961446454646, 1.6171364212058155,
        1.0426876258069404, 1.4890645441069434, 1.524360197848974,
        3.647616189048395, -0.13091124354621334, 3.00908924504507,
        3.0535529001007755, 0.5630443683890836, 0.017344017201497187,
        0.6357631951812274],
       [2.3336889043921905, 1.8581842283293701, 1.1863448057043966,
        0.03330767030920721, 0.24750038906130714, 6.7199864817391814,
        3.4116204676084645, -0.03440022513071406, 0.8620100944961367,
        6.205311799745012, 2.485464215178989, 0.888467423101732,
        1.5073812810524347, 5.17963024762655, 0.013341009520840887,
        1.1669711799626767, 0.4098917274492859, 1.4321070529297375,
        2.8067943179102604, 0.04059964096676245, 3.465729542085774,
        2.734376005711116, 0.7550164232185884, 0.33174910293973353,
        0.7426144749177271],
       [1.2984400981118553, 1.7387655777152178, 1.9698031589114953,
        2.4734082532330013, -0.35639106900673445, 4.2399956281953015,
        5.2437351113867665, 0.26136528834463646, 2.0393730308186355,
        2.9403722506476804, -0.038281209005648235, -0.15211808029258023,
        2.1620748115844024, 3.9459850755559653, 1.8826430906462392,
        1.023927050736279, 0.03016225451979514, 2.0743177211345163,
        2.6595247994742537, 1.5968938858825796, 0.37790868837008945,
        8.952396975738205, 0.625116472395171, -0.13200145151850762,
        0.9433295886048575],
       [1.404266312439557, 1.8485823079748689, 0.9674956103070456,
        3.5207385453479803, 0.07784853271625514, 2.657578602218755,
        -0.0305630541792917, 0.055676437752941246, 1.9245607793672315,
        4.763481579043452, 0.13296350417201508, 1.5139359814727869,
        1.5624737972211835, 6.125956551715964, 4.941671845084829,
        0.8131962863277721, 0.1288501538849027, 0.936633984948289,
        2.209494602909036, 1.2652515742093096, 4.5112115006130935,
        2.7471172764932588, 0.6728523509695268, 0.12074586891316277,
        0.5994895899596933],
       [1.3522845464400937, 1.8273876140652976, 0.30580342511948677,
        3.573029830301148, 0.18164323594022552, 3.517163844895624,
        0.12166908124619465, 1.6726158584724917, 1.3610190464964622,
        4.294513373054747, 1.9335629566506503, 1.3355680171342883,
        1.1128215032241766, 3.822232233812159, 0.7572439544410863,
        1.4883977558977015, 0.7193900167139979, 1.5941321686765817,
        3.266511693104473, 1.388870520149735, 5.073332493675952,
        4.30804158621414, 0.6287655038552828, 0.07947854862674554,
        0.7628753626543481]], dtype=object)

# print coarse optimization info

In [11]:
logging.info(f'_________________________________end coarse optimization stage_________________________________')
logging.info(f"Stage1: Final best morphology: {best_morphology}, Fitness: {best_fitness}, best_efficiency: {best_efficiency}, best reward function: {best_rewardfunc}, Material cost: {best_material}, Reward: {best_reward}")
logging.info(f'folder_name:{folder_name}')
logging.info(f'parameter_list:{parameter_list}')
logging.info(f'fitness_matrix:{fitness_matrix}')
logging.info(f'efficiency_matrix:{efficiency_matrix}')
logging.info(f'_________________________________enter fine optimization stage_________________________________')

# configuration of fine optimization

In [55]:
# 获取矩阵中所有非 None 的值和它们的坐标
all_values_with_coords = []
for i in range(len(efficiency_matrix_select)):
    for j in range(len(efficiency_matrix_select[0])):
        value = efficiency_matrix_select[i][j]
        if value is not None:
            all_values_with_coords.append(((i, j), value))

# 按值降序排序
sorted_values = sorted(all_values_with_coords, key=lambda x: x[1], reverse=True)

# 计算前 20% 的数量（至少选1个）
top_k = max(1, int(len(sorted_values) * 0.05))
# 取前 20% 个坐标
efficiency_coarse_best = [coord for coord, val in sorted_values[:top_k]]

logging.info(f"fitness_coarse_best {efficiency_coarse_best}")
logging.info(f"fitness_coarse_best values {sorted_values[:top_k]}")


In [56]:

coarse_best = efficiency_coarse_best
coarse_best

[(2, 6), (2, 21), (0, 18), (3, 14), (4, 18), (1, 6)]

# enter fine optimization stage

In [57]:
final_optimized_results = []  # 用来记录每个 coarse_best 的最优结果
for rewardfunc_index, morphology_index in coarse_best:
    
    morphology = morphology_list[morphology_index]
    parameter = parameter_list[morphology_index]
    rewardfunc = rewardfunc_list[rewardfunc_index]
    
    best_efficiency = efficiency_matrix_select[rewardfunc_index][morphology_index]
    best_fitness = fitness_matrix[rewardfunc_index][morphology_index]
    best_morphology = morphology
    best_parameter = parameter
    best_rewardfunc = rewardfunc
    best_material = compute_hopper_volume(parameter)
    
    
    logging.info(f"Initial morphology:{morphology}")
    logging.info(f"Initial parameter:{parameter}" )
    logging.info(f"Initial rewardfunc:{rewardfunc}" )
    logging.info(f"Initial fitness:{best_fitness}" )
    logging.info(f"Initial efficiency:{best_efficiency}" )
    iteration = 0
    print(f"Initial parameter:{parameter}")
    while True:
        improved = False  # 标记是否有改进，方便控制循环

        designer = DGA()
        
         # -------- 优化 morphology --------
        iteration +=1
        improved_morphology, improved_parameter = designer.improve_morphology(
            best_parameter,
            parameter_list,
            efficiency_matrix_select[rewardfunc_index, :],
            folder_name,
            rewardfunc_index, 
            morphology_index,
            iteration
            
        )
        
        print("improved parameter", improved_parameter)
        shutil.copy(improved_morphology, "GPTHopper.xml")
        shutil.copy(best_rewardfunc, "GPTrewardfunc.py")
        
        import GPTrewardfunc
        importlib.reload(GPTrewardfunc)  # 重新加载模块
        from GPTrewardfunc import _get_rew
        GPTHopperEnv._get_rew = _get_rew
        try:
            model_path = Train(morphology_index, rewardfunc_index, folder_name, stage='fine', total_timesteps=5e5)
            improved_fitness, _ = Eva(model_path)
        except Exception as e:
            print(f"Error evaluating design: {e}")
            continue
        improved_material = compute_hopper_volume(improved_parameter)
        improved_efficiency = improved_fitness / improved_material

        if improved_efficiency > best_efficiency:

            best_fitness = improved_fitness
            best_morphology = improved_morphology
            best_parameter = improved_parameter
            best_material = improved_material
            best_efficiency = improved_efficiency
            improved = True

            logging.info(f"Morphology optimization improved iteration {iteration}: material={improved_material}, fitness={improved_fitness}, efficiency={improved_efficiency}")

        # -------- 没有进一步改进，跳出循环 --------
        if not improved:
            logging.info("Not improved Morphology!")
            logging.info("____________________________________________")
            improved = False
            # break
            
            
        # -------- 优化 reward function --------
        iteration +=1
        improved_rewardfunc = designer.improve_rewardfunc(
            best_rewardfunc,
            rewardfunc_list,
            efficiency_matrix_select[:, morphology_index],
            folder_name,
            rewardfunc_index, 
            morphology_index,
            iteration
        )

        shutil.copy(best_morphology, "GPTHopper.xml")
        shutil.copy(improved_rewardfunc, "GPTrewardfunc.py")
        
        import GPTrewardfunc
        importlib.reload(GPTrewardfunc)  # 重新加载模块
        from GPTrewardfunc import _get_rew
        GPTHopperEnv._get_rew = _get_rew
        try:
            model_path = Train(morphology_index, rewardfunc_index, folder_name, stage='fine', total_timesteps=5e5)
            improved_fitness, _ = Eva(model_path)
            improved_material = compute_hopper_volume(best_parameter)
            improved_efficiency = improved_fitness / improved_material
            print("improved_fitness", improved_fitness)
        except Exception as e:
            print(f"Error evaluating design: {e}")
            continue

        if improved_efficiency > best_efficiency:
            best_fitness = improved_fitness
            best_rewardfunc = improved_rewardfunc
            best_material = improved_material
            best_efficiency = improved_efficiency
            improved = True

            logging.info(f"Reward optimization improved iteration {iteration}: material={improved_material}, fitness={improved_fitness}, efficiency={improved_efficiency}")
        
        if not improved:
            logging.info("Not improved Reward!")
            logging.info("____________________________________________")
            break
                   
    # 保存当前 coarse_best 的最终最优结果
    final_optimized_results.append({
        "best_morphology": best_morphology,
        "best_parameter": best_parameter,
        "best_rewardfunc": best_rewardfunc,
        "best_fitness": best_fitness,
        "best_material": best_material,
        "best_efficiency": best_efficiency,
        "best_iteration":iteration
    })
    logging.info("____________________________________________")
    logging.info(f"Final optimized result: rewardfunc_index{rewardfunc_index} morphology_index{morphology_index}")
    logging.info(f"  Morphology: {best_morphology}")
    logging.info(f"  Parameter: {best_parameter}")
    logging.info(f"  Rewardfunc: {best_rewardfunc}")
    logging.info(f"  Fitness: {best_fitness}")
    logging.info(f"  Material: {best_material}")
    logging.info(f"  Efficiency: {best_efficiency}")
    logging.info("____________________________________________")

Initial parameter:[1.0, 0.8, 0.5, 0.2, 0.18, -0.2, 0.03, 0.02, 0.02, 0.03]
[1.0, 0.7, 0.5, 0.3, 0.3, -0.3, 0.02, 0.03, 0.04, 0.02]
Successfully saved GPTHopper_refine_2_6_1.xml
improved parameter [1.0, 0.7, 0.5, 0.3, 0.3, -0.3, 0.02, 0.03, 0.04, 0.02]
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
[{'role': 'system', 'content': 'You are a reinforcement learning reward function designer'}, {'role': 'user', 'content': '\nYou are a reward engineer trying to write reward functions to solve reinforcement learning tasks as effectively as possible.\nYour goal is to write a reward function for the enviroment that will help the agent learn the task described in text.\n\nTask Description: The hopper is a two-dimensional one-legged figure consisting of four 

In [ ]:

logging.info(f"{final_optimized_results}")

# logging.info(f"fine optimization end: best material cost: {best_material}  fitness: {improved_fitness} merterial_efficiency: {improved_material_efficiency}")

In [58]:
final_optimized_results

[{'best_morphology': 'results/Div_m25_r5/assets/GPTHopper_6.xml',
  'best_parameter': [1.0, 0.8, 0.5, 0.2, 0.18, -0.2, 0.03, 0.02, 0.02, 0.03],
  'best_rewardfunc': 'results/Div_m25_r5/env/GPTrewardfunc_2.py',
  'best_fitness': 5.2437351113867665,
  'best_material': 0.0026871089163704696,
  'best_efficiency': 1951.4412234802828,
  'best_iteration': 2},
 {'best_morphology': 'results/Div_m25_r5/assets/GPTHopper_21.xml',
  'best_parameter': [2.0, 1.0, 0.6, 0.1, 0.15, -0.15, 0.01, 0.02, 0.03, 0.05],
  'best_rewardfunc': 'results/Div_m25_r5/env/GPTrewardfunc_2.py',
  'best_fitness': 8.952396975738205,
  'best_material': 0.005261120497211707,
  'best_efficiency': 1701.614129629383,
  'best_iteration': 2},
 {'best_morphology': 'results/Div_m25_r5/assets/GPTHopper_refine_0_18_3.xml',
  'best_parameter': [1.6,
   1.2,
   0.9,
   0.4,
   0.25,
   -0.25,
   0.015,
   0.025,
   0.035,
   0.03],
  'best_rewardfunc': 'results/Div_m25_r5/env/GPTrewardfunc_0.py',
  'best_fitness': 7.838311330001222,
 

In [10]:
# robodesign best

morphology = "results/Div_m25_r5/assets/GPTHopper_refine_1_13_0.xml"
rewardfunc = "results/Div_m25_r5/env/GPTrewardfunc_1.py"
morphology_index=999
rewardfunc_index=999
parameter = [1.75, 1.15, 0.75, 0.25, 0.2, -0.2, 0.01, 0.025, 0.035, 0.025]


xml_file = hopper_design(parameter)  
# filename = f"GPTHopper.xml"
# file_path = os.path.join(folder_name, "assets", filename)
file_path = "results/Div_m25_r5/assets/GPTHopper_refine_1_13_0.xml"
with open(file_path, "w") as fp:
    fp.write(xml_file)
    
shutil.copy(morphology, "GPTHopper.xml")
shutil.copy(rewardfunc, "GPTrewardfunc.py")
    
import GPTrewardfunc
importlib.reload(GPTrewardfunc)  # 重新加载模块
from GPTrewardfunc import _get_rew
GPTHopperEnv._get_rew = _get_rew


# model_path = Train(morphology_index, rewardfunc_index, folder_name, stage='fine', total_timesteps=1e6)
model_path = f"results/Div_m25_r5/fine/SAC_morphology{morphology_index}_rewardfunc{rewardfunc_index}_3000000.0steps"
# fitness, _ = Eva(model_path)

fitness, _ = Eva_with_qpos_logging(model_path, run_steps=100, video = True, rewardfunc_index=rewardfunc_index, morphology_index=morphology_index)

material = compute_hopper_volume(parameter)
efficiency = fitness / material

logging.info("3e6 steps train\n")
logging.info(f"best_fitness:{fitness}")
logging.info(f"best_efficiency:{efficiency}")
print("3e6 steps train\n")
print(f"best_fitness:{fitness}")
print(f"best_efficiency:{efficiency}")

Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Run 10
Run 11
Run 12
Run 13
Run 14
Run 15
Run 16
Run 17
Run 18
Run 19
Run 20
Run 21
Run 22
Run 23
Run 24
Run 25
Run 26
Run 27
Run 28
Run 29
Run 30
Run 31
Run 32
Run 33
Run 34
Run 35
Run 36
Run 37
Run 38
Run 39
Run 40
Run 41
Run 42
Run 43
Run 44
Run 45
Run 46
Run 47
Run 48
Run 49
Run 50
Run 51
Run 52
Run 53
Run 54
Run 55
Run 56
Run 57
Run 58
Run 59
Run 60
Run 61
Run 62
Run 63
Run 64
Run 65
Run 66
Run 67
Run 68
Run 69
Run 70
Run 71
Run 72
Run 73
Run 74
Run 75
Run 76
Run 77
Run 78
Run 79
Run 80
Run 81
Run 82
Run 83
Run 84
Run 85
Run 86
Run 87
Run 88
Run 89
Run 90
Run 91
Run 92
Run 93
Run 94
Run 95
Run 96
Run 97
Run 98
Run 99
Saved qpos log to /root/autodl-tmp/Hopper/qpos.txt
Average Fitness: 13.9960, Average Reward: 1011118.1367
3e6 steps train

best_fitness:13.995967962826994
best_efficiency:3500.5670266732122


In [6]:
# robodesign best

morphology = "results/Div_m25_r5/assets/GPTHopper_refine_1_13_0.xml"
rewardfunc = "results/Div_m25_r5/env/GPTrewardfunc_1.py"



morphology_index=999
rewardfunc_index=999
parameter = [1.75, 1.15, 0.75, 0.25, 0.2, -0.2, 0.01, 0.025, 0.035, 0.025]

shutil.copy(morphology, "GPTHopper.xml")
shutil.copy(rewardfunc, "GPTrewardfunc.py")

import GPTrewardfunc
importlib.reload(GPTrewardfunc)  # 重新加载模块
from GPTrewardfunc import _get_rew
GPTHopperEnv._get_rew = _get_rew


# model_path = Train(morphology_index, rewardfunc_index, folder_name, stage='fine', total_timesteps=1e6)
model_path = f"results/Div_m25_r5/fine/SAC_morphology{morphology_index}_rewardfunc{rewardfunc_index}_1000000.0steps"
fitness, _ = Eva(model_path)
# fitness, _ = Eva_with_qpos_logging(model_path, run_steps=100, video = True, rewardfunc_index=rewardfunc_index, morphology_index=morphology_index)

best_material = compute_hopper_volume(best_parameter)
best_efficiency = fitness / best_material

logging.info("3e6 steps train\n")
logging.info(f"best_fitness:{fitness}")
logging.info(f"best_efficiency:{best_efficiency}")
print("3e6 steps train\n")
print(f"best_fitness:{fitness}")
print(f"best_efficiency:{best_efficiency}")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
3e6 steps train

best_fitness:1.1902941458717156
best_efficiency:297.70748619512165


2025-04-07 02:15:56,188 - Initial morphology:results/Div_m25_r5/assets/GPTHopper_6.xml
2025-04-07 02:15:56,188 - Initial parameter:[1.0, 0.8, 0.5, 0.2, 0.18, -0.2, 0.03, 0.02, 0.02, 0.03]
2025-04-07 02:15:56,188 - Initial rewardfunc:results/Div_m25_r5/env/GPTrewardfunc_2.py
2025-04-07 02:15:56,188 - Initial fitness:5.2437351113867665
2025-04-07 02:15:56,188 - Initial efficiency:1951.4412234802828

In [9]:
# coarse only best

morphology = "results/Div_m25_r5/assets/GPTHopper_6.xml"
rewardfunc = "results/Div_m25_r5/env/GPTrewardfunc_2.py"

morphology_index=777
rewardfunc_index=777
parameter = [1.0, 0.8, 0.5, 0.2, 0.18, -0.2, 0.03, 0.02, 0.02, 0.03]

shutil.copy(morphology, "GPTHopper.xml")
shutil.copy(rewardfunc, "GPTrewardfunc.py")

import GPTrewardfunc
importlib.reload(GPTrewardfunc)  # 重新加载模块rewardfunc
from GPTrewardfunc import _get_rew
GPTHopperEnv._get_rew = _get_rew

model_path = Train(morphology_index, rewardfunc_index, folder_name, stage='fine', total_timesteps=1e6)
# model_path = f"results/Div_m25_r5/fine/SAC_morphology{morphology_index}_rewardfunc{rewardfunc_index}_1000000.0steps"
# fitness, _ = Eva(model_path)
fitness, _ = Eva_with_qpos_logging(model_path, run_steps=100, video = True, rewardfunc_index=rewardfunc_index, morphology_index=morphology_index)

material = compute_hopper_volume(parameter)
efficiency = fitness / material

logging.info("coarse only best 3e6 steps train\n")
logging.info(f"fitness:{fitness}")
logging.info(f"efficiency:{efficiency}")
print("3e6 steps train\n")
print(f"fitness:{fitness}")
print(f"efficiency:{efficiency}")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
3e6 steps train

fitness:7.887110028279259
efficiency:2935.1657389952516


In [4]:
# human


morphology = "results/Div_m25_r5/assets/GPTHopper_50.xml"
rewardfunc = "results/Div_m25_r5/env/GPTrewardfunc_10.py"

morphology_index=888
rewardfunc_index=888
parameter = [1.45, 1.06, 0.6, 0.1, -0.13, 0.26, 0.05, 0.05, 0.04, 0.06]

shutil.copy(morphology, "GPTHopper.xml")
shutil.copy(rewardfunc, "GPTrewardfunc.py")

import GPTrewardfunc
importlib.reload(GPTrewardfunc)  # 重新加载模块
from GPTrewardfunc import _get_rew
GPTHopperEnv._get_rew = _get_rew

# model_path = Train(morphology_index, rewardfunc_index, folder_name, stage='fine', total_timesteps=1e6)
model_path = f"results/Div_m25_r5/fine/SAC_morphology{morphology_index}_rewardfunc{rewardfunc_index}_1000000.0steps"
# fitness, _ = Eva(model_path)
fitness, _ = Eva_with_qpos_logging(model_path, run_steps=100, video = True, rewardfunc_index=rewardfunc_index, morphology_index=morphology_index)


material = compute_hopper_volume(parameter)
efficiency = fitness / material

logging.info("human best 3e6 steps train\n")
logging.info(f"fitness:{fitness}")
logging.info(f"efficiency:{efficiency}")
print("human 3e6 steps train\n")
print(f"fitness:{fitness}")
print(f"efficiency:{efficiency}")

Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Run 10
Run 11
Run 12
Run 13
Run 14
Run 15
Run 16
Run 17
Run 18
Run 19
Run 20
Run 21
Run 22
Run 23
Run 24
Run 25
Run 26
Run 27
Run 28
Run 29
Run 30
Run 31
Run 32
Run 33
Run 34
Run 35
Run 36
Run 37
Run 38
Run 39
Run 40
Run 41
Run 42
Run 43
Run 44
Run 45
Run 46
Run 47
Run 48
Run 49
Run 50
Run 51
Run 52
Run 53
Run 54
Run 55
Run 56
Run 57
Run 58
Run 59
Run 60
Run 61
Run 62
Run 63
Run 64
Run 65
Run 66
Run 67
Run 68
Run 69
Run 70
Run 71
Run 72
Run 73
Run 74
Run 75
Run 76
Run 77
Run 78
Run 79
Run 80
Run 81
Run 82
Run 83
Run 84
Run 85
Run 86
Run 87
Run 88
Run 89
Run 90
Run 91
Run 92
Run 93
Run 94
Run 95
Run 96
Run 97
Run 98
Run 99
fitness 2.3020562946661527
Saved qpos log to /root/autodl-tmp/Hopper/qpos.txt
Average Fitness: 2.0766, Average Reward: 398.1811
human 3e6 steps train

fitness:2.076599771478961
efficiency:131.26409682440337


2025-04-07 10:50:19,543 - morphology: 50, rewardfunc: 2, material cost: 0.005429560128926578 reward: 1016.4147318163663 fitness: 3.0493164867488556 efficiency: 561.6139087406524

In [15]:
# Robodesign (w/o Morphology Design)

morphology = "results/Div_m25_r5/assets/GPTHopper_50.xml"
rewardfunc = "results/Div_m25_r5/env/GPTrewardfunc_2.py"

morphology_index=555
rewardfunc_index=555

parameter = [1.45, 1.06, 0.6, 0.1, -0.13, 0.26, 0.05, 0.05, 0.04, 0.06]

shutil.copy(morphology, "GPTHopper.xml")
shutil.copy(rewardfunc, "GPTrewardfunc.py")

import GPTrewardfunc
importlib.reload(GPTrewardfunc)  # 重新加载模块
from GPTrewardfunc import _get_rew
GPTHopperEnv._get_rew = _get_rew

model_path = Train(morphology_index, rewardfunc_index, folder_name, stage='fine', total_timesteps=1e6)
# model_path = f"results/Div_m25_r5/fine/SAC_morphology{morphology_index}_rewardfunc{rewardfunc_index}_3000000.0steps"
fitness, _ = Eva(model_path)
material = compute_hopper_volume(parameter)
efficiency = fitness / material

logging.info("Robodesign (w/o Morphology Design) best 3e6 steps train\n")
logging.info(f"fitness:{fitness}")
logging.info(f"efficiency:{efficiency}")
print("Robodesign (w/o Morphology Design) 3e6 steps train\n")
print(f"fitness:{fitness}")
print(f"efficiency:{efficiency}")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
Robodesign (w/o Morphology Design) 3e6 steps train

fitness:4.233495359427274
efficiency:267.60377825224765


2025-04-07 09:35:14,008 - morphology: 18, rewardfunc: 10, material cost: 0.0022881266493645657 reward: 1471.1455354782527 fitness: 3.783628418579109 efficiency: 1653.5922168599618

In [20]:
# Robodesign (w/o Reward Shaping)

morphology = "results/Div_m25_r5/assets/GPTHopper_18.xml"
rewardfunc = "results/Div_m25_r5/env/GPTrewardfunc_10.py"

morphology_index=444
rewardfunc_index=444

parameter =  [0.9, 0.4, 0.15, 0.05, 0.05, -0.05, 0.02, 0.03, 0.04, 0.01]

shutil.copy(morphology, "GPTHopper.xml")
shutil.copy(rewardfunc, "GPTrewardfunc.py")

import GPTrewardfunc
importlib.reload(GPTrewardfunc)  # 重新加载模块
from GPTrewardfunc import _get_rew
GPTHopperEnv._get_rew = _get_rew

model_path = Train(morphology_index, rewardfunc_index, folder_name, stage='fine', total_timesteps=1e6)
# model_path = f"results/Div_m25_r5/fine/SAC_morphology{morphology_index}_rewardfunc{rewardfunc_index}_3000000.0steps"
fitness, _ = Eva(model_path)
material = compute_hopper_volume(parameter)
efficiency = fitness / material

logging.info("Robodesign (w/o Reward Shaping) best 3e6 steps train\n")
logging.info(f"fitness:{fitness}")
logging.info(f"efficiency:{efficiency}")
print("Robodesign (w/o Reward Shaping) 3e6 steps train\n")
print(f"fitness:{fitness}")
print(f"efficiency:{efficiency}")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
Robodesign (w/o Reward Shaping) 3e6 steps train

fitness:4.457903414345486
efficiency:1948.2765150187327


2025-04-08 00:50:43,757 - Initial morphology:results/noDiv_m25_r5/assets/GPTHopper_16.xml
2025-04-08 00:50:43,757 - Initial parameter:[1.5, 1.1, 0.7, 0.3, 0.4, -0.1, 0.06, 0.04, 0.03, 0.02]
2025-04-08 00:50:43,757 - Initial rewardfunc:results/noDiv_m25_r5/env/GPTrewardfunc_2.py
2025-04-08 00:50:43,757 - Initial fitness:1.0491079684899782
2025-04-08 00:50:43,757 - Initial efficiency:1222.0645575053077

In [17]:
# Robodesign (w/o Diversity Reflection)

morphology = "results/noDiv_m25_r5/assets/GPTHopper_16.xml"
rewardfunc = "results/noDiv_m25_r5/env/GPTrewardfunc_2.py"

morphology_index=333
rewardfunc_index=333

parameter =  [1.5, 1.1, 0.7, 0.3, 0.4, -0.1, 0.06, 0.04, 0.03, 0.02]

shutil.copy(morphology, "GPTHopper.xml")
shutil.copy(rewardfunc, "GPTrewardfunc.py")

import GPTrewardfunc
importlib.reload(GPTrewardfunc)  # 重新加载模块
from GPTrewardfunc import _get_rew
GPTHopperEnv._get_rew = _get_rew

model_path = Train(morphology_index, rewardfunc_index, folder_name, stage='fine', total_timesteps=1e6)
# model_path = f"results/Div_m25_r5/fine/SAC_morphology{morphology_index}_rewardfunc{rewardfunc_index}_3000000.0steps"
fitness, _ = Eva(model_path)
material = compute_hopper_volume(parameter)
efficiency = fitness / material

logging.info("Robodesign (w/o Diversity Reflection) best 3e6 steps train\n")
logging.info(f"fitness:{fitness}")
logging.info(f"efficiency:{efficiency}")
print("Robodesign (w/o Diversity Reflection) 3e6 steps train\n")
print(f"fitness:{fitness}")
print(f"efficiency:{efficiency}")


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
Robodesign (w/o Diversity Reflection) 3e6 steps train

fitness:4.726470799983718
efficiency:491.66090927902854


2025-04-09 04:00:22,222 - iteration:2, morphology: 0, rewardfunc: 2, material cost: 0.015820013405927 reward: 1428.1315493516208 fitness: 3.4351751639953476 efficiency: 217.14110322487792

In [5]:
# eureka reward

morphology = "results/Div_m25_r5/assets/GPTHopper_50.xml"
rewardfunc = "results/eureka/env/GPTrewardfunc_2_2.py"


morphology_index=222
rewardfunc_index=222

parameter = [1.45, 1.06, 0.6, 0.1, -0.13, 0.26, 0.05, 0.05, 0.04, 0.06]

shutil.copy(morphology, "GPTHopper.xml")
shutil.copy(rewardfunc, "GPTrewardfunc.py")

import GPTrewardfunc
importlib.reload(GPTrewardfunc)  # 重新加载模块
from GPTrewardfunc import _get_rew
GPTHopperEnv._get_rew = _get_rew

# model_path = Train(morphology_index, rewardfunc_index, folder_name, stage='fine', total_timesteps=1e6)
model_path = f"results/Div_m25_r5/fine/SAC_morphology{morphology_index}_rewardfunc{rewardfunc_index}_1000000.0steps"
# fitness, _ = Eva(model_path)
fitness, _ = Eva_with_qpos_logging(model_path, run_steps=100, video = True, rewardfunc_index=rewardfunc_index, morphology_index=morphology_index)

material = compute_hopper_volume(parameter)
efficiency = fitness / material

logging.info(" eureka reward best 3e6 steps train\n")
logging.info(f"fitness:{fitness}")
logging.info(f"efficiency:{efficiency}")
print(" eureka reward 3e6 steps train\n")
print(f"fitness:{fitness}")
print(f"efficiency:{efficiency}")


Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Run 10
Run 11
Run 12
Run 13
Run 14
Run 15
Run 16
Run 17
Run 18
Run 19
Run 20
Run 21
Run 22
Run 23
Run 24
Run 25
Run 26
Run 27
Run 28
Run 29
Run 30
Run 31
Run 32
Run 33
Run 34
Run 35
Run 36
Run 37
Run 38
Run 39
Run 40
Run 41
Run 42
Run 43
Run 44
Run 45
Run 46
Run 47
Run 48
Run 49
Run 50
Run 51
Run 52
Run 53
Run 54
Run 55
Run 56
Run 57
Run 58
Run 59
Run 60
Run 61
Run 62
Run 63
Run 64
Run 65
Run 66
Run 67
Run 68
Run 69
Run 70
Run 71
Run 72
Run 73
Run 74
Run 75
Run 76
Run 77
Run 78
Run 79
Run 80
Run 81
Run 82
Run 83
Run 84
Run 85
Run 86
Run 87
Run 88
Run 89
Run 90
Run 91
Run 92
Run 93
Run 94
Run 95
Run 96
Run 97
Run 98
Run 99
fitness 2.5589989584737585
Saved qpos log to /root/autodl-tmp/Hopper/qpos.txt
Average Fitness: 2.1738, Average Reward: 410.9972
 eureka reward 3e6 steps train

fitness:2.1737909311192776
efficiency:137.40765417460787


In [ ]:

morphology = "results/eureka_morphology/assets/GPTHopper_6.xml"
rewardfunc = "results/eureka_morphology/env/GPTrewardfunc_0.py"

morphology_index=111
rewardfunc_index=111

parameter = [0.7, 0.42, 0.22, 0.09, -0.05, 0.09, 0.035, 0.035, 0.025, 0.015]

shutil.copy(morphology, "GPTHopper.xml")
shutil.copy(rewardfunc, "GPTrewardfunc.py")

import GPTrewardfunc
importlib.reload(GPTrewardfunc)  # 重新加载模块
from GPTrewardfunc import _get_rew
GPTHopperEnv._get_rew = _get_rew

# model_path = Train(morphology_index, rewardfunc_index, folder_name, stage='fine', total_timesteps=1e6)
model_path = f"results/Div_m25_r5/fine/SAC_morphology{morphology_index}_rewardfunc{rewardfunc_index}_1000000.0steps"
# fitness, _ = Eva(model_path)
fitness, _ = Eva_with_qpos_logging(model_path, run_steps=100, video = True, rewardfunc_index=rewardfunc_index, morphology_index=morphology_index)

material = compute_hopper_volume(parameter)
efficiency = fitness / material

logging.info(" eureka reward best 3e6 steps train\n")
logging.info(f"fitness:{fitness}")
logging.info(f"efficiency:{efficiency}")
print(" eureka reward 3e6 steps train\n")
print(f"fitness:{fitness}")
print(f"efficiency:{efficiency}")


Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Run 10
Run 11
Run 12
Run 13
Run 14
Run 15
Run 16
Run 17
Run 18
Run 19
Run 20
Run 21
Run 22
Run 23
Run 24
Run 25
Run 26
Run 27
Run 28
Run 29
Run 30
Run 31
Run 32
Run 33
Run 34
Run 35
Run 36
Run 37
Run 38
Run 39
Run 40
Run 41
Run 42
Run 43
Run 44
Run 45
Run 46
Run 47
Run 48
Run 49
Run 50
Run 51
Run 52
Run 53
Run 54
Run 55
Run 56
Run 57
Run 58
Run 59
Run 60
Run 61
Run 62
Run 63
Run 64
Run 65
Run 66
Run 67
Run 68
Run 69
Run 70
Run 71
Run 72
Run 73
Run 74
Run 75
Run 76
Run 77
Run 78
Run 79
Run 80
Run 81
Run 82
Run 83
Run 84
Run 85
Run 86
Run 87
